In [1]:
from nltk.tokenize import sent_tokenize
from os import listdir
from os.path import isfile, join

In [2]:
path = '/kaggle/input/building-materials/'
files = [file for file in listdir(path) if isfile(join(path, file))]

In [3]:
pretraining_data = ''
for filename in files:
    with open(path + filename, 'r') as file:
        full_text = file.read()
        sentences = sent_tokenize(full_text)
        pretraining_data += '\n'.join(sentences) + '\n\n'

In [4]:
with open('/kaggle/working/pretraining_data.txt', 'w') as file:
    file.write(pretraining_data)

In [5]:
!pip install tensorflow==1.15

     |████████████████████████████████| 412.3 MB 18 kB/s s eta 0:00:01
     |████████████████████████████████| 50 kB 180 kB/s  eta 0:00:01
     |████████████████████████████████| 3.8 MB 22.2 MB/s eta 0:00:01
     |████████████████████████████████| 503 kB 44.9 MB/s eta 0:00:01
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7539 sha256=1f16dacaa2338a34fad599cc438e7a994dda8eec14ce2e366e27c3a115f41624
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
ERROR: tensorflow-probability 0.10.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Suc

In [6]:
cd ../input/bertsrc/

/kaggle/input/bertsrc


In [7]:
!ls

CONTRIBUTING.md		    optimization_test.py
LICENSE			    predicting_movie_reviews_with_bert_on_tf_hub.ipynb
README.md		    requirements.txt
__init__.py		    run_classifier.py
create_pretraining_data.py  run_classifier_with_tfhub.py
extract_features.py	    run_pretraining.py
modeling.py		    run_squad.py
modeling_test.py	    sample_text.txt
multilingual.md		    tokenization.py
optimization.py		    tokenization_test.py


In [8]:
!python create_pretraining_data.py \
    --input_file=/kaggle/working/pretraining_data.txt \
    --output_file=/kaggle/working/tf_examples.tfrecord \
    --vocab_file=/kaggle/input/sentence-ru-cased-l12-h768-a12/vocab.txt \
    --do_lower_case=True \
    --max_seq_length=128 \
    --max_predictions_per_seq=20 \
    --masked_lm_prob=0.15 \
    --random_seed=12345 \
    --dupe_factor=5

W1111 11:51:36.401466 139695582779200 module_wrapper.py:139] From create_pretraining_data.py:437: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W1111 11:51:36.401715 139695582779200 module_wrapper.py:139] From create_pretraining_data.py:437: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

W1111 11:51:36.402036 139695582779200 module_wrapper.py:139] From /kaggle/input/bertsrc/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W1111 11:51:37.085612 139695582779200 module_wrapper.py:139] From create_pretraining_data.py:444: The name tf.gfile.Glob is deprecated. Please use tf.io.gfile.glob instead.

W1111 11:51:37.086758 139695582779200 module_wrapper.py:139] From create_pretraining_data.py:446: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

I1111 11:51:37.089318 139695582779200 create_pretraining_data.py:4

I1111 11:51:37.377014 139695582779200 create_pretraining_data.py:166] Wrote 110 total instances


In [9]:
import tensorflow as tf
AUTO = tf.data.experimental.AUTOTUNE

In [10]:
!python run_pretraining.py \
  --input_file=/kaggle/working/tf_examples.tfrecord \
  --output_dir=/kaggle/working/pretraining_output \
  --do_train=True \
  --do_eval=True \
  --bert_config_file=/kaggle/input/sentence-ru-cased-l12-h768-a12/bert_config.json \
  --init_checkpoint=/kaggle/input/sentence-ru-cased-l12-h768-a12/bert_model.ckpt \
  --train_batch_size=32 \
  --max_seq_length=128 \
  --max_predictions_per_seq=20 \
  --num_train_steps=1 \
  --num_warmup_steps=10 \
  --learning_rate=2e-5

W1111 11:51:48.958547 140120507623232 module_wrapper.py:139] From run_pretraining.py:407: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W1111 11:51:48.958783 140120507623232 module_wrapper.py:139] From run_pretraining.py:407: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

W1111 11:51:48.959130 140120507623232 module_wrapper.py:139] From /kaggle/input/bertsrc/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W1111 11:51:48.967796 140120507623232 module_wrapper.py:139] From run_pretraining.py:414: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.

W1111 11:51:48.968508 140120507623232 module_wrapper.py:139] From run_pretraining.py:418: The name tf.gfile.Glob is deprecated. Please use tf.io.gfile.glob instead.

W1111 11:51:48.969546 140120507623232 module_wrapper.py:139] From run_pretraining.py:420: The name tf.logg

W1111 11:51:50.952824 140120507623232 deprecation.py:506] From /kaggle/input/bertsrc/modeling.py:358: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W1111 11:51:50.968776 140120507623232 deprecation.py:323] From /kaggle/input/bertsrc/modeling.py:671: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.Dense instead.
W1111 11:51:50.970072 140120507623232 deprecation.py:323] From /opt/conda/lib/python3.7/site-packages/tensorflow_core/python/layers/core.py:187: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
W1111 11:51:53.394193 140120507623232 module_wrapper.py:139] From run_

W1111 11:51:54.324723 140120507623232 deprecation.py:323] From /opt/conda/lib/python3.7/site-packages/tensorflow_core/python/ops/math_grad.py:1375: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
I1111 11:52:01.610058 140120507623232 estimator.py:1150] Done calling model_fn.
I1111 11:52:01.612006 140120507623232 basic_session_run_hooks.py:541] Create CheckpointSaverHook.
I1111 11:52:05.207699 140120507623232 monitored_session.py:240] Graph was finalized.
2020-11-11 11:52:05.208245: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2020-11-11 11:52:05.216056: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2000170000 Hz
2020-11-11 11:52:05.216361: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x556e7e

I1111 11:56:35.359511 140120507623232 run_pretraining.py:173]   name = bert/encoder/layer_9/output/dense/kernel:0, shape = (3072, 768), *INIT_FROM_CKPT*
I1111 11:56:35.359616 140120507623232 run_pretraining.py:173]   name = bert/encoder/layer_9/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
I1111 11:56:35.359714 140120507623232 run_pretraining.py:173]   name = bert/encoder/layer_9/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
I1111 11:56:35.359821 140120507623232 run_pretraining.py:173]   name = bert/encoder/layer_9/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
I1111 11:56:35.359974 140120507623232 run_pretraining.py:173]   name = bert/encoder/layer_10/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
I1111 11:56:35.360138 140120507623232 run_pretraining.py:173]   name = bert/encoder/layer_10/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
I1111 11:56:35.360285 140120507623232 run_pretraining.py:173]   name = bert/encoder/

I1111 11:58:50.159387 140120507623232 evaluation.py:167] Evaluation [80/100]
I1111 11:59:05.803962 140120507623232 evaluation.py:167] Evaluation [90/100]
I1111 11:59:22.602163 140120507623232 evaluation.py:167] Evaluation [100/100]
I1111 11:59:22.765046 140120507623232 evaluation.py:275] Finished evaluation at 2020-11-11-11:59:22
I1111 11:59:22.765399 140120507623232 estimator.py:2049] Saving dict for global step 10: global_step = 10, loss = 9.023505, masked_lm_accuracy = 0.051591065, masked_lm_loss = 8.419887, next_sentence_accuracy = 0.88625, next_sentence_loss = 0.6000477
I1111 11:59:23.313268 140120507623232 estimator.py:2109] Saving 'checkpoint_path' summary for global step 10: /kaggle/working/pretraining_output/model.ckpt-10
I1111 11:59:23.314178 140120507623232 error_handling.py:101] evaluation_loop marked as finished
I1111 11:59:23.314461 140120507623232 run_pretraining.py:483] ***** Eval results *****
I1111 11:59:23.314592 140120507623232 run_pretraining.py:485]   global_step 